In [1]:
# 앞의 함수에서는 목표 변수를 제외한 나머지 변수를 고정시키기 위해 tmp1과 tmp2의 서로 다른 함수를 정의했지만,
# 이 두 변수에 대해 편미분을 동시에 계산하고 싶다면 어떻게 해야하는가?

# 양쪽의 편미분을 묶어서 계산한다. 이처럼 모든 변수의 편미분을 벡터로 정리한 것을 기울기gradient라고 한다.

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같고, 요소가 모두 0인 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h) # (f(x+h) - f(x-h)) / (2*h)
        x[idx] = tmp_val # 값 복원

    return grad        

In [3]:
import numpy as np

def function_2(x):
    return x[0]**2 + x[1]**2

# 기울기
print(numerical_gradient(function_2, np.array([3.0, 4.0]))) # 실제로는 미세한 오차가 있지만, numpy 배열을 출력할 때 자동으로 보기 좋게 가공됨
print(numerical_gradient(function_2, np.array([0.0, 2.0])))
print(numerical_gradient(function_2, np.array([3.0, 0.0])))

# 기울기는 각 지점에서 낮아지는 방향을 가리킨다.
# 기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다.

[6. 8.]
[0. 4.]
[6. 0.]


In [4]:
# 그러나 기울기가 가리키는 곳에 정말 함수의 최솟값이 있는지(즉, 그 쪽이 정말로 나아가야 할 방향인지)는 보장할 수 없다.
# 경사법은 기울기가 0인 장소를 찾지만, 그것이 최솟값인지, 극솟값이나 안장점인지는 알 수 없다. 또, 복잡하고 찌그러진 모양의 함수라면 '고원'이라 하는 정체기에 빠질수도 있다.

# 경사법 : 현 위치에서 기울어진 방향으로 일정 거리 이동하여 다시 기울기를 구하고, 이 과정을 반복하여 함수의 값을 점차 줄이는 방법. 기계학습 최적화에 흔히 쓰인다.
# 학습률learning rate : 갱신하는 양. 신경망 학습에서, 한 번의 학습으로 얼마만큼 학습해야 할지, 매개변수 값을 얼마나 갱신하느냐를 정하는 것
# 경사법에는 최솟값을 찾는 경사 하강법과 최댓값을 찾는 경사 상승법이 있다.

In [5]:
# 경사 하강법

def gradient_descent(f, init_x, lr=0.01, step_num=100): # init_x : 초깃값, lr : 학습률, step_num : 경사법에 따른 반복 학습 횟수
    x = init_x
    
    for _ in range(step_num): # 갱신처리를 step_num번 반복
        grad = numerical_gradient(f,x) # 함수의 기울기 구하기
        x -= lr * grad #기울기에 학습률을 곱한 값
    return x

In [6]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100) # 거의 (0,0)에 가까운 결과.

array([-6.11110793e-10,  8.14814391e-10])

In [10]:
# 학습률이 너무 큰 예 : lr=10.0
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100))
# --> 너무 큰 값으로 발산

# 학습률이 너무 작은 예 : lr=1e-10
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100))
# --> 거의 갱신되지 않은채 끝남

# 학습률과 같은 매개변수를 하이퍼파라미터hyper paremeter,초매개변수 라고 하며, 가중치와 편향같은 데이터에 의해 '자동'으로 획득되는 매개변수와 달리 사람이 '직접' 설정해야 하는 매개변수이다.

[-2.58983747e+13 -1.29524862e+12]
[-2.99999994  3.99999992]


In [14]:
# 신경망을 통해 기울기를 구하는 코드

import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화, shape가 2 * 3인 가중치 매개변수 W를 가짐
    
    # 예측 수행 함수
    def predict(self, x):
        return np.dot(x, self.W)
    
    # 손실 함수의 값을 구하는 함수
    def loss(self, x, t): # x : 입력 데이터, t : 정답 레이블
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [16]:
net = simpleNet()
print(net.W) # 가중치 매개변수

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
np.argmax(p) # 최댓값의 인덱스

t = np.array([0, 0, 1]) # 정답레이블
net.loss(x,t)

[[-0.04693469 -0.00801594 -1.36829273]
 [-0.03183881 -0.76193313 -0.10780165]]
[-0.05681574 -0.69054939 -0.91799712]


1.5306855344564876

In [18]:
# net.W를 받아 손실 함수를 계산하는 새로운 함수
def f(W): # 인수 W는 numerical_gradient(f,x) 내부의 f(x)와의 일관성을 위해 만든 더미dummy, 사용하지 않음
    return net.loss(x, t)

# 간단히 람다로 구현하면 이렇게 구현할 수 있음
f = lambda w: net.loss(x,t)

dW = numerical_gradient(f, net.W) # shape는 2*3의 2차원 배열
print(dW)

[[ 0.30717714  0.16299034 -0.47016748]
 [ 0.46076571  0.24448551 -0.70525122]]
